# ランダムフォレスト分類器(Random Forest Classifier)
Model Definition: アンサンブル学習を用いた決定木の集合体で、高い汎化性能を持つランダムフォレスト分類器を使用。

Training: サンプリングされたトレーニングデータのサブセットを用いて、複数の決定木を並行して学習。

Prediction: 学習された決定木の多数決により、検証セットでの予測を実施。

Evaluation: ランダムフォレストモデルの性能を分析するため、分類レポートと精度指標を表示。また、特徴量の重要度分析も可能。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Define dataset filenames
train_file = "train_1_split.csv"
val_file = "val_1_split.csv"

# Construct dynamic paths based on the current working directory
current_dir = os.getcwd()
train_path = os.path.join(current_dir, "/content/drive/MyDrive/image_analytics/data/features", train_file)
val_path = os.path.join(current_dir, "/content/drive/MyDrive/image_analytics/data/features", val_file)

# Optional check if paths exist
if not os.path.isfile(train_path):
    print(f"Warning: {train_path} が見つかりません")
if not os.path.isfile(val_path):
    print(f"Warning: {val_path} が見つかりません")

# Load the training and validation datasets
train = pd.read_csv(train_path)
val = pd.read_csv(val_path)

# Separate features and labels for the training and validation sets
X_train = train.iloc[:, 2:].astype(np.float32)
y_train = train['label']
X_val = val.iloc[:, 2:].astype(np.float32)
y_val = val['label']

# Sample a subset for training
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=50000, random_state=42, stratify=y_train)

# Confirm dimensions
print("Training Features Shape:", X_train.shape)
print("Training Labels Shape:", y_train.shape)
print("Validation Features Shape:", X_val.shape)
print("Validation Labels Shape:", y_val.shape)


## Random Forest Classifier: Training, Prediction, and Evaluation

ここでは、ランダムフォレスト分類器を実装し、検証データでその性能を評価します。ランダムフォレストは、
複数の決定木を組み合わせたアンサンブル学習手法で、高い予測精度と過学習への耐性が特徴です。

1. **必要なライブラリのインポート**:

2. **ランダムフォレスト分類器の定義**:
   - RandomForestClassifierは、デフォルトで100個の決定木を生成
   - 再現性を維持するために`random_state=42`を指定。
   - デフォルトのパラメータでは、各ノードの分割に特徴量の平方根個の特徴量をランダムに選択

3. **モデルのトレーニング**:
   - 各決定木は、ブートストラップサンプリングされたデータセットで学習
   - 訓練データからランダムに選ばれた特徴量のサブセットを使用して分岐を決定

4. **予測の実行**:
   - 学習済みの分類器を使用して、検証データセット（X_val）に対し予測。これらの予測（rf_y_pred）は、検証セットの各サンプルに対する分類器の予測を提供する。

5. **モデルの性能評価**:
   - モデルの精度を計算し、これは正しい予測の割合を反映。
   - 分類レポートは、各クラスの適合率、再現率、F1スコアなどの詳細な指標を提供し、異なるクラス間でのモデルの性能を理解するのに役立ちます。



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Define the Random Forest Classifier
rf_clf = RandomForestClassifier(random_state=42)

# Train the classifier on the sampled training data
rf_clf.fit(X_train_sample, y_train_sample)

# Make predictions on the validation data
rf_y_pred = rf_clf.predict(X_val)

# Evaluate the model
print("Random Forest Classifier Accuracy:", accuracy_score(y_val, rf_y_pred))
print("\nClassification Report:\n", classification_report(y_val, rf_y_pred))


In [4]:
import joblib

# Define the new path for saving the model inside the models folder
model_save_path = "/content/drive/MyDrive/image_analytics/models/random_forest_classifier.pkl"

# Save the trained Random Forest classifier
joblib.dump(rf_clf, model_save_path)

print(f"Random Forest model saved to {model_save_path}.")


Random Forest model saved to /content/drive/MyDrive/image_analytics/models/random_forest_classifier.pkl.


### Model Evaluation Metrics

モデルを評価するための包括的な指標のセットを計算し、表示します。

1. **精度（**ACCURACY**）**: 正しい予測の全体的な割合。
2. **適合率（**PRECISION**）（加重）**: クラスの不均衡を考慮した、全クラスの適合率の加重平均。
3. **再現率（**RECALL**）（加重）**: 全クラスの再現率の加重平均。
4. **F1スコア（**F1 SCORE**）（加重）**: 適合率と再現率のバランスを取る、全クラスのF1スコアの加重平均。


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Step 1: Calculate basic metrics
accuracy = accuracy_score(y_val, rf_y_pred)
precision = precision_score(y_val, rf_y_pred, average='weighted')
recall = recall_score(y_val, rf_y_pred, average='weighted')
f1 = f1_score(y_val, rf_y_pred, average='weighted')

# Step 2: Display the classification report
print("Classification Report:")
print(classification_report(y_val, rf_y_pred))

# Step 3: Display all calculated metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision (Weighted): {precision:.2f}")
print(f"Recall (Weighted): {recall:.2f}")
print(f"F1 Score (Weighted): {f1:.2f}")



## 可視化
1. **混同行列**: 真のラベルと予測ラベルを表示し、誤分類を理解するのに役立ちます。
2. **特徴量重要度プロット**: モデルにおける最も重要な特徴量を表示し、上位10個の特徴量に焦点を当てます。
3. **各クラスのROC曲線**: マルチクラス設定における各クラスのROC曲線とAUCスコアを提供し、各クラスのパフォーマンスを確認できます。
4. **予測確率の分布**: 特定のクラスに対するモデルの確信度を視覚化します。他のクラスを分析する場合は`class_index`を調整してください。


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np

# Step 1: Confusion Matrix
conf_matrix = confusion_matrix(y_val, rf_y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_val), yticklabels=np.unique(y_val))
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Step 2: Feature Importance Plot
importances = rf_clf.feature_importances_
indices = np.argsort(importances)[-10:]  # Show top 10 features for simplicity
plt.figure(figsize=(10, 8))
plt.barh(range(len(indices)), importances[indices], align='center')
plt.yticks(range(len(indices)), [f"Feature {i}" for i in indices])
plt.xlabel("Relative Importance")
plt.title("Top 10 Feature Importances in Random Forest")
plt.show()

# Step 3: ROC Curve for each class (multi-class ROC)
fpr = {}
tpr = {}
roc_auc = {}

for i in np.unique(y_val):
    fpr[i], tpr[i], _ = roc_curve(y_val == i, rf_clf.predict_proba(X_val)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in np.unique(y_val):
    plt.plot(fpr[i], tpr[i], label=f"Class {i} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Each Class")
plt.legend(loc="best")
plt.show()

# Step 4: Distribution of Predicted Probabilities for a Sample Class
class_index = 0  # Change this to another class to see different distributions
predicted_probabilities = rf_clf.predict_proba(X_val)[:, class_index]
plt.figure(figsize=(10, 8))
sns.histplot(predicted_probabilities, bins=30, kde=True)
plt.title(f"Distribution of Predicted Probabilities for Class {class_index}")
plt.xlabel("Predicted Probability")
plt.ylabel("Frequency")
plt.show()
